In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
database_name = 'Colorado'

# search settings
area = False
zip_code = False
radius = False # can be 0, 5, 10, 25, 50, or 100
state = 'CO'

In [ ]:
url = "https://forms.nabip.org/consumer/findagent2.cfm" 
# options = webdriver.ChromeOptions() #newly added 
# options.headless = True #newly added 
# with webdriver.Chrome(options=options) as browser: #modified 
# 	browser.get(url)
# 	print("Page URL:", browser.current_url) 
# 	print("Page Title:", browser.title)

browser = webdriver.Chrome()
browser.get(url)


# enter zip code
if zip_code:
    browser.find_element(By.XPATH, '/html/body/div/div[3]/form/table/tbody/tr[2]/td[2]/input').send_keys(zip_code)

# enter radius
if radius:
    radius_select = Select(browser.find_element(By.XPATH, '/html/body/div/div[3]/form/table/tbody/tr[2]/td[2]/select'))
    radius_select.select_by_visible_text('Single Zip Code') 
    radius_select.select_by_value(radius)

if state:
    state_select = Select(browser.find_element(By.XPATH, '/html/body/div/div[3]/form/table/tbody/tr[8]/td[2]/select'))
    state_select.select_by_visible_text('Choose a State/Province') 
    state_select.select_by_value(state)


# search database
browser.find_element(By.XPATH, '/html/body/div/div[3]/form/table/tbody/tr[2]/td[2]/input').send_keys(Keys.ENTER)

In [ ]:
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

# find total num of brokers in area
total_brokers = soup.find_all('p')

# check there are results
if len(total_brokers) == 1:
    print("No members found")
    exit()

total_brokers = int(total_brokers[1].text.split()[4])

In [ ]:
brokers = []

# loop until all brokers are found
while len(brokers) < total_brokers:

    # extract broker info
    new_brokers = soup.find('div', {'class': 'contentContainer'})

    new_brokers = new_brokers.find_all('div')

    for broker in new_brokers:
        data_dict = {'Name':'', 'Company':'', 'Website':'', 'Work Phone':'', 'Fax':'', 'Email':'', \
                    'Address':'', 'Zip Code':'', 'Certifications':'', 'Certified States':'', 'Practice Areas':''}

        broker_text = broker.text.split('\n')
        broker_text = list(filter(None, broker_text))

        # make sure its a full contact div
        if len(broker_text) < 5:
                continue
        
        broker_text.pop(0) # remove vCard item

        # extract broker info to data_dict
        for index, item in enumerate(broker_text):
            if index == 0:
                data_dict['Name'] = broker_text[index]
            elif 'Certifications' in item:
                data_dict['Certifications'] = broker_text[index]
            elif 'Work Phone' in item:
                temp_text = broker_text[index].split('Work Phone: ')[1]
                if 'Fax' in temp_text:
                    temp_text = temp_text.split(' Fax: ')
                    data_dict['Work Phone'] = temp_text[0]
                    data_dict['Fax'] = temp_text[1]
                else:
                    data_dict['Work Phone'] = temp_text
            elif 'Chapter' in item:
                data_dict['Certified States'] = broker_text[index].split('Chapter: ')[1]
            elif 'Web Site' in item:
                data_dict['Website'] = broker_text[index].split('Web Site: ')[1]
            elif 'E-mail' in item:
                temp_text = broker_text[index].split('E-mail: ')[1]
                if 'Practice Areas' in temp_text:
                    temp_text = temp_text.split(' Practice Areas: ')
                    data_dict['Email'] = temp_text[0]
                    data_dict['Practice Areas'] = temp_text[1]
                else:
                    data_dict['Email'] = temp_text
                
            else:
                data_dict['Address'] = broker_text[index]
                data_dict['Zip Code'] = broker_text[index].split(' ')[-1]
                
                # get company from address
                split = broker_text[index].split()
                curr_string = ''
                for string in split:
                    if string.isnumeric():
                        break
                    elif string == "PO":
                        break
                    curr_string += string + ' '
                
                data_dict['Company'] = curr_string


        # only add broker if not already in list
        if data_dict not in brokers:
            brokers.append(data_dict)


    print(f'Found {len(brokers)}/{total_brokers} brokers')

    # reload page & create new BS object after all processing
    browser.find_element(By.CLASS_NAME, 'bluebutton').send_keys(Keys.ENTER)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')

browser.close()
print(f'Done!')

In [ ]:
# convert dictionary into DataFrame and export to csv

brokers_final = pd.DataFrame(brokers)

brokers_final.to_csv(f'Output\\{database_name}_brokers.csv', index=False)